In [1]:
import pandas as pd
from sqlalchemy import create_engine, URL, text
import re
from bs4 import BeautifulSoup
from tqdm import tqdm
tqdm.pandas()

In [ ]:
url_object = URL.create(
    drivername="",
    username="",
    password="",
    host="",
    port="",
    database=""
)
engine = create_engine(url_object)

# Create Indices on Query Columns (only required once!)

In [3]:
# query_ep_index = """CREATE INDEX ep_index ON ep_fulltext_data(epo_publn_nr, publn_auth);"""
# query_us_brf_summary_index = """CREATE INDEX us_brf_summary_index ON us_brf_summary(patent_id, publn_auth);"""
# query_us_claims_index = """CREATE INDEX us_claims_index ON us_claims(patent_id, publn_auth);"""
# query_us_description_index = """CREATE INDEX us_description_index ON us_description(patent_id, publn_auth);"""

In [4]:
# with engine.connect() as connection:
#     connection.execute(text(query_ep_index))
#     connection.execute(text(query_us_brf_summary_index))
#     connection.execute(text(query_us_claims_index))
#     connection.execute(text(query_us_description_index))

# Extract Cleantech Patents from PATSTAT (only granted and EP or US patents)

## CPC Y02 Classification

In [5]:
# query_cpc_index = """CREATE INDEX idx_cpc_class_symbol ON tls224_appln_cpc(cpc_class_symbol);""" # only required once

In [3]:
query_cpc_y02 = """
    SELECT a.appln_id, a.cpc_class_symbol
    FROM tls224_appln_cpc a
    JOIN tls201_appln b ON a.appln_id = b.appln_id
    WHERE (a.cpc_class_symbol LIKE 'Y02A%' OR
           a.cpc_class_symbol LIKE 'Y02B%' OR
           a.cpc_class_symbol LIKE 'Y02C%' OR
           a.cpc_class_symbol LIKE 'Y02D%' OR
           a.cpc_class_symbol LIKE 'Y02E%' OR
           a.cpc_class_symbol LIKE 'Y02P%' OR
           a.cpc_class_symbol LIKE 'Y02T%' OR
           a.cpc_class_symbol LIKE 'Y02W%')
      AND b.appln_auth IN ('EP', 'US')
      AND b.granted = 'Y';
"""

In [4]:
with engine.connect() as connection:
    # connection.execute(text(query_cpc_index)) # Only required once
    df_cpc = pd.read_sql_query(text(query_cpc_y02), connection)

In [5]:
df = df_cpc.copy()

# Subsume Patents per appln_id

In [6]:
df_grouped = df.groupby('appln_id').agg({
    'cpc_class_symbol': lambda x: list(x)
}).reset_index()

# Extract Patent Fulltext Data for appln_id

## Create Temp Table with appln_id

In [7]:
appln_ids = df_grouped['appln_id'].tolist()

# Create a temporary table and insert application IDs
temp_table_query = """
    DROP TABLE IF EXISTS temp_appln_ids;
    CREATE TEMP TABLE temp_appln_ids (appln_id TEXT);
"""
insert_query = text("INSERT INTO temp_appln_ids (appln_id) VALUES (:appln_id)")

with engine.connect() as connection:
    connection.execute(text(temp_table_query))
    for id in appln_ids:
        connection.execute(insert_query, {'appln_id': id})
    connection.commit()

    # Perform the join query to extract all valid publn_nr per appln_id
    join_query = """
        SELECT t1.appln_id, t2.publn_auth, t2.publn_nr, t2.publn_date, t2. pat_publn_id
        FROM temp_appln_ids t1
        JOIN (
            SELECT appln_id, publn_auth, publn_nr, publn_date, pat_publn_id
            FROM tls211_pat_publn
        ) t2 ON t1.appln_id = t2.appln_id;
    """
    result = connection.execute(text(join_query))
    df_temp_publn = pd.DataFrame(result.fetchall(), columns=result.keys())

df_temp_publn['publn_nr'] = df_temp_publn['publn_nr'].astype(str)
shortest_publn_nr_idx = df_temp_publn.groupby('appln_id')['publn_nr'].apply(lambda x: x.str.len().idxmin())
df_shortest_publn = df_temp_publn.loc[shortest_publn_nr_idx]

# Insert the filtered results back into the database
temp_table_with_publn_query = """
    DROP TABLE IF EXISTS temp_appln_ids_with_publn;
    CREATE TEMP TABLE temp_appln_ids_with_publn (appln_id TEXT, publn_auth TEXT, publn_nr TEXT, publn_date DATE, pat_publn_id TEXT);
    CREATE INDEX temp_appln_ids_with_publn_index ON temp_appln_ids_with_publn(publn_nr, publn_auth);
"""
insert_filtered_query = text("INSERT INTO temp_appln_ids_with_publn (appln_id, publn_auth, publn_nr, publn_date, pat_publn_id) VALUES (:appln_id, :publn_auth, :publn_nr, :publn_date, :pat_publn_id)")

with engine.connect() as connection:
    connection.execute(text(temp_table_with_publn_query))
    for _, row in df_shortest_publn.iterrows():
        connection.execute(insert_filtered_query, {'appln_id': row['appln_id'], 'publn_auth': row['publn_auth'], 'publn_nr': row['publn_nr'], 'publn_date': row['publn_date'], 'pat_publn_id': row['pat_publn_id']})
    connection.commit()

In [13]:
len(df_grouped), len(df_shortest_publn)

(770168, 770168)

In [11]:
df_grouped.head()

,appln_id,cpc_class_symbol
0,101891,[Y02P 20/52]
1,101897,[Y02E 60/50]
2,101926,"[Y02B 10/10, Y02B 10/20, Y02B 10/70, Y02E ..."
3,101976,"[Y02T 10/12, Y02T 10/40]"
4,101984,[Y02D 30/00]


In [12]:
df_shortest_publn.head()

,appln_id,publn_auth,publn_nr,publn_date,pat_publn_id
412135,101891,EP,1360181,2008-10-08,289379223
703992,101897,EP,1333516,2006-12-13,387634414
176854,101926,EP,1711753,2010-08-18,323646172
1067937,101976,EP,2013456,2010-10-13,327969544
1272762,101984,EP,2028791,2012-07-04,365139097


In [ ]:
# Get columns ... from df_shortest_publn and merge with df_grouped

## Extract cited patents

In [8]:
# Query to extract citation relationships
citation_query = """
    SELECT t1.appln_id, t1.publn_auth, t1.publn_nr, t1.publn_date, t1.pat_publn_id, 
           t2.cited_pat_publn_id
    FROM temp_appln_ids_with_publn t1
    JOIN tls212_citation t2 ON t1.pat_publn_id = t2.pat_publn_id
"""

# Execute the query and create DataFrame with citation data
with engine.connect() as connection:
    df_citations = pd.read_sql_query(text(citation_query), connection)
    
# Display the first few rows to verify the data
print(f"Total citations extracted: {len(df_citations)}")

Total citations extracted: 18330656


In [9]:
# Get the unique set of cleantech patent publication IDs
cleantech_pat_publn_ids = set(df_citations['pat_publn_id'].unique())

# Filter to include only citations where both citing and cited patents are cleantech patents
df_cleantech_citations = df_citations[df_citations['cited_pat_publn_id'].isin(cleantech_pat_publn_ids)]

In [10]:
# Create a mapping from pat_publn_id to appln_id using our existing data
pat_to_appln_mapping = dict(zip(df_citations['pat_publn_id'], df_citations['appln_id']))

# Function to map cited_pat_publn_id to cited_appln_id
def get_cited_appln_ids(cited_pat_ids):
    return [pat_to_appln_mapping.get(pat_id) for pat_id in cited_pat_ids if pat_id in pat_to_appln_mapping]

# Aggregate by appln_id
df_cleantech_citations_agg = df_cleantech_citations.groupby('appln_id').agg({
    'publn_auth': 'first',
    'publn_nr': 'first', 
    'publn_date': 'first',
    'pat_publn_id': 'first',
    'cited_pat_publn_id': lambda x: list(x)
}).reset_index()

# Add the cited_appln_id list column
df_cleantech_citations_agg['cited_appln_id'] = df_cleantech_citations_agg['cited_pat_publn_id'].apply(get_cited_appln_ids)

## Extract Title and Abstract

In [23]:
query_title_abstr = """
    SELECT ids.appln_id, ids.publn_nr, ids.publn_auth, title.appln_title, abstract.appln_abstract
    FROM temp_appln_ids_with_publn ids
    JOIN tls202_appln_title title ON ids.appln_id = title.appln_id AND title.appln_title_lg = 'en'
    JOIN tls203_appln_abstr abstract ON ids.appln_id = abstract.appln_id AND abstract.appln_abstract_lg = 'en'
"""

In [24]:
with engine.connect() as connection:
    df_title_abstr = pd.read_sql_query(text(query_title_abstr), connection)

## Extract US Claims

In [25]:
with engine.connect() as connection:
    query_us_claims = """
        SELECT t1.*, t2.*
        FROM temp_appln_ids_with_publn t1
        JOIN us_claims t2 ON t1.publn_nr = t2.patent_id AND t1.publn_auth = t2.publn_auth;
    """
    df_us_claims = pd.read_sql(query_us_claims, connection)

### Preprocess Claims

In [26]:
df_us_claims = df_us_claims.dropna(subset=['claim_text'])
df_us_claims.sort_values(by=['patent_id', 'claim_sequence'], inplace=True)
df_us_claims_grouped = df_us_claims.groupby('appln_id').agg({
    'patent_id': 'first',
    'claim_text': list
}).reset_index()

In [27]:
def process_row(row):
    claim_fulltext = ' '.join(re.sub(r'^\d+\.\s', ' ', text) for text in row['claim_text'])
    return pd.Series({'claim_fulltext': claim_fulltext})
df_us_claims_grouped['claim_fulltext'] = df_us_claims_grouped.apply(process_row, axis=1)
df_us_claims_grouped.drop('claim_text', axis=1, inplace=True)

## Extract EP Fulltext (Title, Abstract, Brf Summary, Claims, Description) - (currently only Title, Abstract, Claims considered)

In [28]:
with engine.connect() as connection:    
    query_ep_fulltext_data = """
        SELECT t1.*, t2.*
        FROM temp_appln_ids_with_publn t1 
        JOIN ep_fulltext_data t2 
            ON t1.publn_nr = t2.epo_publn_nr
            AND t1.publn_auth = t2.publn_auth
        WHERE t2.appln_lng = 'en'
          AND t2.appln_comp = 'CLAIM';
    """
    df_ep_fulltext_data = pd.read_sql(query_ep_fulltext_data, connection)

In [29]:
order = {'B9': 11, 'B8': 10, 'B3': 9, 'B2': 8, 'B1': 7, 'A9': 6, 'A8': 5, 'A4': 4, 'A3': 3, 'A2': 2, 'A1': 1}
df_ep_fulltext_data['order'] = df_ep_fulltext_data['appln_kind'].map(order)
df_ep_fulltext_data.drop(['epo_publn_nr', 'appln_auth', 'appln_date', 'appln_lng', 'appln_text_type'], axis=1, inplace=True)

In [30]:
df_ep_fulltext_data_claims = df_ep_fulltext_data[df_ep_fulltext_data['appln_comp'] == 'CLAIM']

In [31]:
df_ep_fulltext_data_claims = df_ep_fulltext_data_claims.sort_values(by=['publn_nr', 'order'])
df_ep_fulltext_data_claims = df_ep_fulltext_data_claims.groupby('publn_nr').first().reset_index()

### Clean Text

In [36]:
def clean_html(text):
    cleaned_text = re.sub(r'<!--.*?-->', ' ', text)
    soup = BeautifulSoup(cleaned_text, 'html.parser')
    cleaned_text = soup.get_text(separator=' ')
    return cleaned_text
def clean_claim_text(claim_text):
    # Remove all instances of <!--(.*?)-->
    claim_text = re.sub(r'<!--.*?-->', ' ', claim_text)

    # Parse the claim_text as XML using BeautifulSoup
    soup = BeautifulSoup(claim_text, 'html.parser')
    
    # Extract all text from <claim-text> tags
    cleaned_texts = [elem.get_text() for elem in soup.find_all('claim-text') if elem.get_text()]

    # Join the cleaned texts
    cleaned_text = ' '.join(cleaned_texts)
    
    return cleaned_text.strip()

In [37]:
df_ep_fulltext_data_claims['appln_text'] = df_ep_fulltext_data_claims['appln_text'].apply(clean_claim_text)

In [38]:
df_ep_fulltext_data_claims = df_ep_fulltext_data_claims.loc[:, ~df_ep_fulltext_data_claims.columns.duplicated()]

# Build one common dataframe

In [64]:
df_combined = pd.merge(df_grouped, df_cleantech_citations_agg, on='appln_id', how='left')

In [66]:
# Select only needed columns from df_combined
df_combined_subset = df_combined[['appln_id', 'cpc_class_symbol', 'publn_date', 
                                 'pat_publn_id', 'cited_pat_publn_id', 'cited_appln_id']]

# Select only needed columns from df_title_abstr
df_title_abstr_subset = df_title_abstr[['appln_id', 'publn_nr', 'publn_auth', 
                                        'appln_title', 'appln_abstract']]

# Merge the subsets on appln_id
df_combined = df_combined_subset.merge(df_title_abstr_subset, on='appln_id', how='left')

In [67]:
# Merge with df_us_claims_grouped on appln_id
df_combined = df_combined.merge(df_us_claims_grouped, on='appln_id', how='left')

In [68]:
df_ep_fulltext_data_claims_subset = df_ep_fulltext_data_claims[['appln_id', 'appln_text']].copy()
# Merge with df_ep_fulltext_data_claims_subset on appln_id
df_combined = df_combined.merge(df_ep_fulltext_data_claims_subset, on='appln_id', how='left')

In [18]:
# Merge columns claim_fulltext and appln_text
df_combined['claim_fulltext'] = df_combined['claim_fulltext'].fillna(df_combined['appln_text'])
df_combined.drop('appln_text', axis=1, inplace=True)

In [ ]:
# Merge with df_shortest_publn so that all column entries are filled
df_combined_subset = df_combined[['appln_id', 'cpc_class_symbol', 'cited_pat_publn_id', 'cited_appln_id', 'appln_title', 'appln_abstract', 'claim_fulltext']].copy()
df_combined = pd.merge(df_combined_subset, df_shortest_publn[['appln_id', 'publn_nr', 'publn_auth', 'publn_date', 'pat_publn_id']], on='appln_id', how='outer')

In [31]:
df_combined = df_combined[['appln_id', 'publn_nr', 'publn_auth', 'publn_date', 'pat_publn_id', 'cpc_class_symbol', 'cited_pat_publn_id', 'cited_appln_id', 'appln_title', 'appln_abstract', 'claim_fulltext']].copy()

In [50]:
df_combined.to_parquet('/mnt/hdd02/Projekt_EDV_TEK/edv_tek_cleantech_patstat_diffusion.parquet', index=False)

# Statistical Measures for Diffusion

In [36]:
# Calculate the number of patents with citations (for US and EP separately)
us_patents_with_citations = df_combined[(df_combined['publn_auth'] == 'US') & 
                                        (df_combined['cited_pat_publn_id'].notna())].shape[0]
print(f"Number of US patents with citations: {us_patents_with_citations}")

ep_patents_with_citations = df_combined[(df_combined['publn_auth'] == 'EP') &
                                        (df_combined['cited_pat_publn_id'].notna())].shape[0]
print(f"Number of EP patents with citations: {ep_patents_with_citations}")

Number of US patents with citations: 381540
Number of EP patents with citations: 8085


In [47]:
# Filter for US patents
us_patents = df_combined[df_combined['publn_auth'] == 'US']

# Create a mapping from appln_id to publn_auth
appln_id_to_auth = dict(zip(df_combined['appln_id'], df_combined['publn_auth']))

# Initialize counters
us_to_ep_citations = 0
us_patents_with_ep_citations = 0

# Iterate through US patents
for _, us_patent in tqdm(us_patents.iterrows()):
    cited_appln_ids = us_patent['cited_appln_id']
    
    # Skip if there are no citations - fixed condition
    if cited_appln_ids is None or isinstance(cited_appln_ids, float) and pd.isna(cited_appln_ids) or (isinstance(cited_appln_ids, list) and len(cited_appln_ids) == 0):
        continue
    
    # Count EP citations in this patent
    ep_citations_in_patent = 0
    for app_id in cited_appln_ids:
        if app_id in appln_id_to_auth and appln_id_to_auth[app_id] == 'EP':
            ep_citations_in_patent += 1
    
    us_to_ep_citations += ep_citations_in_patent
    if ep_citations_in_patent > 0:
        us_patents_with_ep_citations += 1

# Print results
print(f"Total US patents analyzed: {len(us_patents)}")
print(f"US patents citing at least one EP patent: {us_patents_with_ep_citations}")
print(f"Total citations from US to EP patents: {us_to_ep_citations}")

# Calculate statistics
if len(us_patents) > 0:
    print(f"Percentage of US patents citing EP patents: {us_patents_with_ep_citations/len(us_patents)*100:.2f}%")
    
if us_patents_with_ep_citations > 0:
    print(f"Average number of EP citations per US patent (that cites EP): {us_to_ep_citations/us_patents_with_ep_citations:.2f}")

587781it [00:17, 33039.96it/s]

Total US patents analyzed: 587781
US patents citing at least one EP patent: 1228
Total citations from US to EP patents: 1363
Percentage of US patents citing EP patents: 0.21%
Average number of EP citations per US patent (that cites EP): 1.11


In [51]:
# Filter for EP patents
ep_patents = df_combined[df_combined['publn_auth'] == 'EP']

# Create a mapping from appln_id to publn_auth
appln_id_to_auth = dict(zip(df_combined['appln_id'], df_combined['publn_auth']))

# Initialize counters
ep_to_us_citations = 0
ep_patents_with_us_citations = 0

# Iterate through EP patents
for _, ep_patent in tqdm(ep_patents.iterrows()):
    cited_appln_ids = ep_patent['cited_appln_id']
    
    # Skip if there are no citations - fixed condition
    if cited_appln_ids is None or isinstance(cited_appln_ids, float) and pd.isna(cited_appln_ids) or (isinstance(cited_appln_ids, list) and len(cited_appln_ids) == 0):
        continue
    
    # Count US citations in this patent
    us_citations_in_patent = 0
    for app_id in cited_appln_ids:
        if app_id in appln_id_to_auth and appln_id_to_auth[app_id] == 'US':
            us_citations_in_patent += 1
    
    ep_to_us_citations += us_citations_in_patent
    if us_citations_in_patent > 0:
        ep_patents_with_us_citations += 1

# Print results
print(f"Total EP patents analyzed: {len(ep_patents)}")
print(f"EP patents citing at least one US patent: {ep_patents_with_us_citations}")
print(f"Total citations from EP to US patents: {ep_to_us_citations}")

# Calculate statistics
if len(ep_patents) > 0:
    print(f"Percentage of EP patents citing US patents: {ep_patents_with_us_citations/len(ep_patents)*100:.2f}%")
    
if ep_patents_with_us_citations > 0:
    print(f"Average number of US citations per EP patent (that cites US): {ep_to_us_citations/ep_patents_with_us_citations:.2f}")

182387it [00:04, 38205.03it/s]

Total EP patents analyzed: 182387
EP patents citing at least one US patent: 8035
Total citations from EP to US patents: 12892
Percentage of EP patents citing US patents: 4.41%
Average number of US citations per EP patent (that cites US): 1.60


In [55]:
# Handle both string and datetime formats
if isinstance(df_combined['publn_date'].iloc[0], str):
    # Convert string dates to datetime
    date_series = pd.to_datetime(df_combined['publn_date'])
else:
    # Already datetime format
    date_series = df_combined['publn_date']

# Extract years from the datetime objects
years = date_series.dt.year.dropna()

# Find the min and max years
if not years.empty:
    earliest_year = int(years.min())
    latest_year = int(years.max())
    
    print(f"Earliest publication year: {earliest_year}")
    print(f"Latest publication year: {latest_year}")
    print(f"Time span: {latest_year - earliest_year + 1} years")
else:
    print("No valid dates found in the dataset.")

# Optional: Show distribution of patents by year
year_counts = years.value_counts().sort_index()
print("\nNumber of patents by year:")
print(year_counts)

Earliest publication year: 1836
Latest publication year: 2023
Time span: 188 years

Number of patents by year:
publn_date
1836        1
1837        6
1838        5
1839        3
1840        5
        ...  
2019    51237
2020    49168
2021    42646
2022    35828
2023     5126
Name: count, Length: 188, dtype: int64


In [53]:
type(df_combined['publn_date'][0])

str

In [54]:
df_combined['publn_date'][0]

'2008-10-08'

In [52]:
df_combined.head()

,appln_id,publn_nr,publn_auth,publn_date,pat_publn_id,cpc_class_symbol,cited_pat_publn_id,cited_appln_id,appln_title,appln_abstract,claim_fulltext
0,101891,1360181,EP,2008-10-08,289379223,[Y02P 20/52],None,None,EPOXIDATION CATALYST AND PROCESS,In a process for the production of an oxirane ...,A process for the epoxidation bf an olefin by ...
1,101897,1333516,EP,2006-12-13,387634414,[Y02E 60/50],None,None,Fuel cell and process for improving the transp...,The method involves generating pressure differ...,Method for improving a heat and mass transfer ...
2,101926,1711753,EP,2010-08-18,323646172,"[Y02B 10/10, Y02B 10/20, Y02B 10/70, Y02E ...",None,None,"VACUUM-INSULATED, MODULAR ROOF SYSTEM","The invention relates to an insulated, modular...",Insulated modular roof system (100) comprising...
3,101976,2013456,EP,2010-10-13,327969544,"[Y02T 10/12, Y02T 10/40]",None,None,DEVICE FOR SUPPLYING A REDUCING AGENT INTO AN ...,The invention relates to a device (2) for supp...,Device for supplying a reducing agent into an ...
4,101984,2028791,EP,2012-07-04,365139097,[Y02D 30/00],None,None,Recognition of incorrect configurations on net...,The method involves providing information of n...,Method for recognizing incorrect configuration...
